# Profiling CUDA in Torch

In [1]:
!pip install ninja

In [2]:
!sudo apt update
!sudo apt install build-essential ninja-build

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease

Building dependency tree... Done
Reading state information... Done
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/ongoing/gpu-mode/gpu-mode-notes/lec1-profiling-cuda-in-torch/

/content/drive/MyDrive/ongoing/gpu-mode/gpu-mode-notes/lec1-profiling-cuda-in-torch


## TL; DR

当需要定制化 cuda kernel 并且整合到 torch 代码中时，我们主要关心两个问题:
1. 如何对 cuda kernel 做 profiling (这也有助于我们决定是否需要自己实现 cuda kernel, 我们的目标是在真正需要的地方自己实现定制化的 kernel, 并且效率高于编译技术生成的实现)
2. 如何在 torch 代码中使用定制化的 cuda kernel.

对于 profiling，主要的工具有：
- cuda event
- torch.autograd.profiler
- torch.profiler
- ncu

对于如何在 torch 中使用定制化的 cuda kernel，主要有两种比较简单的方式：
- 使用 triton 实现 kernel (然后可以直接使用，triton kernel 就是一个 python 函数加了 @triton.jit);
- 使用 torch.utils.cpp_extension 的 load_inline

## Profiling CUDA Kernel in torch

In [5]:
import torch

### CUDA Event

In [6]:
def time_pytorch_function(f, input):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    for _ in range(10):
        # warmup
        f(input)

    start.record()
    f(input)
    end.record()
    torch.cuda.synchronize()

    return start.elapsed_time(end)

In [7]:
t = torch.randn((10000, 10000)).cuda()
time_pytorch_function(torch.square, t) # in milliseconds

3.227776050567627

### torch.autograd.profiler

In [8]:
with torch.autograd.profiler.profile(use_device='cuda') as prof:
    torch.square(t)
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
# the result denotes most task is done in aten::pow

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
             aten::square         0.36%      35.288us         1.74%     168.391us     168.391us      37.000us         1.10%       3.369ms       3.369ms             1  
                aten::pow         0.98%      94.657us         1.30%     125.441us     125.441us       3.320ms        98.55%       3.332ms       3.332ms             1  
        aten::result_type         0.03%       2.590us         0.03%       2.590us       2.590us       7.000us         0.21%       7.000us       7.000us        

### torch.profiler

In [9]:
# ## Default way to use profiler
# with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
#     for _ in range(10):
#         a = torch.square(torch.randn(10000, 10000).cuda())

# prof.export_chrome_trace("trace.json")

## With warmup and skip
# https://pytorch.org/docs/stable/profiler.html

# Non-default profiler schedule allows user to turn profiler on and off
# on different iterations of the training loop;
# trace_handler is called every time a new trace becomes available
def trace_handler(prof):
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("/tmp/test_trace_" + str(prof.step_num) + ".json")

with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],

    # In this example with wait=1, warmup=1, active=2, repeat=1,
    # profiler will skip the first step/iteration,
    # start warming up on the second, record
    # the third and the forth iterations,
    # after which the trace will become available
    # and on_trace_ready (when set) is called;
    # the cycle repeats starting with the next step

    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2,
        repeat=1),
    on_trace_ready=trace_handler
    # on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
    # used when outputting for tensorboard
    ) as p:
        for iter in range(10):
            torch.square(torch.randn(10000, 10000).cuda())
            # send a signal to the profiler that the next iteration has started
            p.step()

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.00%       0.000us         0.00%       0.000us       0.000us     228.241ms       129.41%     228.241ms     114.121ms             2  
                                            aten::copy_         0.01%      96.878us         9.67%     170.636ms      85.318ms     169.957ms        96.37%     169.957ms      84.978ms             2  
         

### ncu: NVIDIA Nsight Compute


In [10]:
# ncu --set full -o output $(which python) test.py

## Integrating Custom kernel

### Triton Kernel

In [11]:
import triton
import triton.language as tl

@triton.jit
def square_kernel(
    x_ptr,
    output_ptr,
    n_elements,
    BLOCK_SIZE: tl.constexpr,
):
  pid = tl.program_id(axis=0)
  block_start = pid * BLOCK_SIZE
  offsets = block_start + tl.arange(0, BLOCK_SIZE)
  x = tl.load(x_ptr + offsets, mask=offsets < n_elements, other=0.0)
  output = x * x
  tl.store(output_ptr + offsets, output, mask=offsets < n_elements)

In [12]:
def square(x):
    n_elements = x.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']),)

    output = torch.empty_like(x)

    square_kernel[grid](
        x,
        output,
        n_elements,
        BLOCK_SIZE=1024,
    )

    return output

In [13]:
square(t)

tensor([[1.0990, 1.7078, 0.3829,  ..., 0.7603, 1.8344, 0.6198],
        [0.0065, 0.2429, 0.0070,  ..., 2.5966, 2.1683, 0.6403],
        [0.8363, 0.5699, 1.4727,  ..., 1.1237, 4.8443, 0.1980],
        ...,
        [0.7112, 1.3919, 0.0086,  ..., 0.5383, 0.7787, 0.0115],
        [0.3561, 0.0340, 0.2762,  ..., 3.7034, 0.0137, 0.2513],
        [0.6512, 0.0190, 0.1780,  ..., 0.8531, 0.2841, 1.0220]],
       device='cuda:0')

### Triton kernel: use torch.compile as starting point

In [31]:
!TORCH_LOGS="output_code" python compiled_square.py

V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] Output code: 
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] # AOT ID: ['0_inference']
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] from ctypes import c_void_p, c_long, c_int
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] import torch
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] import math
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] import random
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] import os
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] import tempfile
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] from math import inf, nan
V0608 06:03:41.240000 50357 torch/_inductor/graph.py:2045] [0/0] [__output_code] from torch._indu

### Load inline

In [14]:
from torch.utils.cpp_extension import load_inline

In [20]:
cpp_source = """
std::string hello() {
    return "Hello World!";
}
"""

In [21]:
!ls
!mkdir -p tmp

ncu_prof.log  notes.ipynb  test.py  tmp


In [22]:
hello_module = load_inline(name='hello_module', cpp_sources=[cpp_source], verbose=True, functions=['hello'], build_directory='./tmp')

The input conditions for extension module hello_module have changed. Bumping to version 3 and re-building as hello_module_v3...
Emitting ninja build file ./tmp/build.ninja...
Building extension module hello_module_v3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module hello_module_v3...


In [23]:
hello_module.hello()

'Hello World!'

## Load inline: continue

In [28]:
cuda_source = """
__global__ void square_kernel(float *x, float * output, int H, int W) {
    int xid = blockIdx.x * blockDim.x + threadIdx.x;
    int yid = blockIdx.y * blockDim.y + threadIdx.y;
    if (xid < W && yid < H) {
        output[yid * W + xid] = x[yid * W + xid] * x[yid * W + xid];
    }
}

torch::Tensor square_matrix(torch::Tensor x) {
    const int height = x.size(0);
    const int width = x.size(1);
    auto result = torch::empty_like(x);
    dim3 threads_per_block(16, 16);
    dim3 number_of_blocks((width + threads_per_block.x - 1) / threads_per_block.x,
                          (height + threads_per_block.y - 1) / threads_per_block.y);
    square_kernel<<<number_of_blocks, threads_per_block>>>(x.data_ptr<float>(), result.data_ptr<float>(), height, width);
    return result;
}
"""

cpp_source = """
torch::Tensor square_matrix(torch::Tensor);
"""

In [26]:
!mkdir -p tmp_cuda

In [29]:
my_module = load_inline(name='my_module', cpp_sources=[cpp_source], cuda_sources=[cuda_source], verbose=True, functions=['square_matrix'], build_directory='./tmp_cuda')

The input conditions for extension module my_module have changed. Bumping to version 1 and re-building as my_module_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file ./tmp_cuda/build.ninja...
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module my_module_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module my_module_v1...


In [30]:
my_module.square_matrix(t)

tensor([[1.0990, 1.7078, 0.3829,  ..., 0.7603, 1.8344, 0.6198],
        [0.0065, 0.2429, 0.0070,  ..., 2.5966, 2.1683, 0.6403],
        [0.8363, 0.5699, 1.4727,  ..., 1.1237, 4.8443, 0.1980],
        ...,
        [0.7112, 1.3919, 0.0086,  ..., 0.5383, 0.7787, 0.0115],
        [0.3561, 0.0340, 0.2762,  ..., 3.7034, 0.0137, 0.2513],
        [0.6512, 0.0190, 0.1780,  ..., 0.8531, 0.2841, 1.0220]],
       device='cuda:0')